In [ ]:
import pandas as pd
from datasets import Dataset, DatasetDict,load_from_disk,load_dataset

In [ ]:
dataset_path = "medal_dataset"
# dataset = load_from_disk(dataset_path)

dataset = load_dataset("lutful2004/MeDAL-dataset")


In [ ]:
from datasets import ClassLabel

# get unique labels
unique_labels = set()
for split in dataset.keys():
    # Convert to list first to make it hashable
    unique_labels.update(dataset[split]["LABEL"])
unique_labels = sorted(unique_labels)

# create ClassLabel feature
label_class = ClassLabel(num_classes=len(unique_labels), names=unique_labels)


In [ ]:
# cast the LABEL column
dataset = dataset.cast_column("LABEL", label_class)

# now you can do stratified sampling
sampled = DatasetDict({
    "train": dataset["train"].train_test_split(
        train_size=0.1, stratify_by_column="LABEL", seed=42
    )["train"],
    "test": dataset["test"].train_test_split(
        train_size=0.1, stratify_by_column="LABEL", seed=42
    )["train"],
    "valid": dataset["valid"].train_test_split(
        train_size=0.1, stratify_by_column="LABEL", seed=42
    )["train"],
})

print(sampled)


In [ ]:
from huggingface_hub import notebook_login

# # Create a smaller version of the dataset for easier upload
# small_medal_dataset = DatasetDict({
#     "train": train_dataset.select(range(10000)),
#     "test": test_dataset.select(range(1000)),
#     "valid": valid_dataset.select(range(1000))
# })

# Import required libraries

# Login to Huggingface Hub
notebook_login()

# Push to hub
sampled.push_to_hub(
    "lutful2004/MeDAL-dataset-small",
    private=False,  # Set to True if you want a private dataset
    token=None  # Will use the token from notebook_login
)

print("Dataset uploaded successfully to Huggingface Hub")